In [39]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from gensim.summarization.summarizer import summarize

In [9]:
df = pd.read_csv('df_with_gensim_summaries.csv')

In [10]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'title', 'author', 'year', 'month', 'category'], axis = 1)

In [11]:
df.head()

,content,len_article,gensim_summary
0,And never more so than in Showtime’s new...,12219,"In the second season finale, back in 1991, the..."
1,AlphaGo’s victory isn’t a defeat for hum...,11068,When speaking to DeepMind and Google developer...
2,How a weapon against war became a weapon...,14088,International visitors for the event are commo...
3,Genius quietly laid off a bunch of its e...,12170,"In a post on the Genius blog at the time, co-f..."
4,Inside the test flight of Facebook’s fir...,11967,But if your goal is to stay in the air for a l...


In [12]:
def create_soup(url):
    facts = requests.get(url)
    soup = BeautifulSoup(facts.content, 'html.parser')
    return soup

In [128]:
def get_article_text():
    url = input('Enter a url: \n')
    soup = create_soup(url)
    
    text = []
    for i in soup.find_all('p'):
        text.append(i.get_text())
        
    full_text = '. '.join(text).replace('\xa0', '')
    word_count_ = full_text.count(' ')
    gensim_summary = summarize(full_text, word_count = word_count_/10).strip('. ').replace('.\n', '')
    
    print('\n Full Text:\n', full_text, '\n\n')
    print('Extractive Summary: \n', gensim_summary)

In [129]:
get_article_text()



Enter a url: 
https://www.yahoo.com/news/dominican-republic-hotel-says-del-164734854.html

 Full Text:
 A Dominican Republic hotel has fought back against a woman's accusations that it did not promptly take care of her in the aftermath of an alleged attack by claiming that she requested $2.2 million in compensation prior to going public with her now-viral story.. A representative from the Majestic Elegance Hotel in Punta Cana reached out to AOL on Wednesdayin an attempt to set the record straight on an incident involvingTammy Lawrence-Daley, who accused the hotel of handlingherinjuries irresponsibly. In a press release, the hotel said it took the proper steps to take care of thevictim andaccused her of trying to make money off the attack.. "Majestic Resorts paid for all hospital expenses and offered Mrs. Lawrence, her husband and her fellow travelers a complimentary extension of their stay at the hotel for all the time necessary to recover and arrange their departure from the country,"